In [13]:
!pip install pyDOE

In [1]:
import tensorflow as tf
import datetime, os
#hide tf logs 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'} 
#0 (default) shows all, 1 to filter out INFO logs, 2 to additionally filter out WARNING logs, and 3 to additionally filter out ERROR logs
import scipy.optimize
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import seaborn as sns 
import codecs, json

# generates same random numbers each time
np.random.seed(1234)
tf.random.set_seed(1234)

print("TensorFlow version: {}".format(tf.__version__))

TensorFlow version: 2.8.0


# *Data Prep*

Training and Testing data is prepared from the solution file

In [198]:
from scipy.stats import multivariate_normal
#getting collocation points
x = np.linspace(-1, 1, 128)                     # 256 points between -1 and 1 [256x1]
y = np.linspace(-1, 1, 128)
t = np.linspace(0, 0.3, 100)                    # 100 time points between 0 and 1 [100x1]
X, Y, T = np.meshgrid(x, y ,t) 
usol=np.zeros((128, 128 ,len(t)))
pos = np.dstack((X[:,:,0], Y[:,:,0]))
rv = multivariate_normal(mean=[0.0 , 0.0], cov=[0.05 , 0.05])
usol[:][:,:,0] = rv.pdf(pos)

#collocation points for every position and every time



# *Test Data*

We prepare the test data to compare against the solution produced by the PINN.

In [199]:
''' X_u_test = [X[i],T[i]] [25600,2] for interpolation'''
X_u_test = np.hstack((X.flatten()[:,None], Y.flatten()[:,None], T.flatten()[:,None]))

# Domain bounds
lb = X_u_test[0]  # [-1. 0.]
ub = X_u_test[-1] # [1.  0.99]

'''
   Fortran Style ('F') flatten,stacked column wise!
   u = [c1 
        c2
        .
        .
        cn]

   u =  [25600x1] 
'''
u = usol.flatten('F')[:,None] 

# *Training Data*

The boundary conditions serve as the test data for the PINN and the collocation points are generated using **Latin Hypercube Sampling**

In [200]:
def trainingdata(N_u,N_f):

    '''Boundary Conditions'''

    #Initial Condition -1 =< x,y =<1 and t = 0  
    initial_xy = np.hstack((np.vstack(X[:,:,0][:,:,None]), np.vstack(Y[:,:,0][:,:,None]) ,np.vstack(T[:,:,0][:,:,None]))) #L1
    initial_u = np.vstack(usol[:][:,:,0][:,:,None])

    #Boundary Condition -1 < x < 1, y = -1 and 0 =< t =<1
    bottomedge_xy = np.hstack((np.tile(X[0,:,0], len(t))[:,None], np.tile(Y[0,:,0], len(t))[:,None], np.hstack((np.tile(T[0,0,:][:,None], 128)))[:,None])) #L2
    bottomedge_u = np.tile(usol[:,-1,-1], len(t))[:,None]

    #Boundary Condition -1 < x < 1, y = 1 and 0 =< t =<1
    topedge_xy = np.hstack((np.tile(X[0,:,0], len(t))[:,None], np.tile(Y[-1,:,0], len(t))[:,None], np.hstack((np.tile(T[0,0,:][:,None], 128)))[:,None])) #L3
    topedge_u = np.tile(usol[:,-1,-1], len(t))[:,None]
    
    #Boundary Condition x = -1, -1 < y < 1 and 0 =< t =<1
    leftedge_xy = np.hstack((np.tile(X[:,0,0], len(t))[:,None], np.tile(Y[:,0,0], len(t))[:,None], np.hstack((np.tile(T[0,0,:][:,None], 128)))[:,None])) #L4
    leftedge_u = np.tile(usol[:,-1,-1], len(t))[:,None]

    #Boundary Condition x = 1, -1 < y < 1 and 0 =< t =<1
    rightedge_xy = np.hstack((np.tile(X[:,-1,0], len(t))[:,None], np.tile(Y[:,0,0], len(t))[:,None], np.hstack((np.tile(T[0,0,:][:,None], 128)))[:,None])) #L5
    rightedge_u = np.tile(usol[:,-1,-1], len(t))[:,None]

    all_X_u_train = np.vstack([initial_xy, bottomedge_xy, topedge_xy, leftedge_xy, rightedge_xy]) # X_u_train [456,2] (456 = 256(L1)+100(L2)+100(L3))
    all_u_train = np.vstack([initial_u, bottomedge_u, topedge_u, leftedge_u, rightedge_u])   #corresponding u [456x1]

    #choose random N_u points for training
    idx = np.random.choice(all_X_u_train.shape[0], N_u, replace=False) 

    X_u_train = all_X_u_train[idx, :] #choose indices from  set 'idx' (x,t)
    u_train = all_u_train[idx,:]      #choose corresponding u

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    X_f_train = lb + (ub-lb)*lhs(3,N_f) 
    X_f_train = np.vstack((X_f_train, X_u_train)) # append training points to collocation points 
    return X_f_train, X_u_train, u_train 


# **PINN**

Generate a **PINN** of L hidden layers, each with n neurons. 

Initialization: ***Xavier***

Activation: *tanh (x)*

In [201]:
class Sequentialmodel(tf.Module): 
    def __init__(self, layers, name=None):
        self.itera = 0
        self.W = []  #Weights and biases
        self.parameters = 0 #total number of parameters
        
        for i in range(len(layers)-1):
            
            input_dim = layers[i]
            output_dim = layers[i+1]
            
            #Xavier standard deviation 
            std_dv = np.sqrt((2.0/(input_dim + output_dim)))

            #weights = normal distribution * Xavier standard deviation + 0
            w = tf.random.normal([input_dim, output_dim], dtype = 'float64') * std_dv
                       
            w = tf.Variable(w, trainable=True, name = 'w' + str(i+1))

            b = tf.Variable(tf.cast(tf.zeros([output_dim]), dtype = 'float64'), trainable = True, name = 'b' + str(i+1))
                    
            self.W.append(w)
            self.W.append(b)
            
            self.parameters +=  input_dim * output_dim + output_dim
    
    def evaluate(self,x):
        
        x = (x-lb)/(ub-lb)
        
        a = x
        
        for i in range(len(layers)-2):
            
            z = tf.add(tf.matmul(a, self.W[2*i]), self.W[2*i+1])
            a = tf.nn.tanh(z)
            
        a = tf.add(tf.matmul(a, self.W[-2]), self.W[-1]) # For regression, no activation to last layer
        return a
    
    def get_weights(self):

        parameters_1d = []  # [.... W_i,b_i.....  ] 1d array
        
        for i in range (len(layers)-1):
            
            w_1d = tf.reshape(self.W[2*i],[-1])   #flatten weights 
            b_1d = tf.reshape(self.W[2*i+1],[-1]) #flatten biases
            
            parameters_1d = tf.concat([parameters_1d, w_1d], 0) #concat weights 
            parameters_1d = tf.concat([parameters_1d, b_1d], 0) #concat biases
        
        return parameters_1d
        
    def set_weights(self,parameters):
                
        for i in range (len(layers)-1):

            shape_w = tf.shape(self.W[2*i]).numpy() # shape of the weight tensor
            size_w = tf.size(self.W[2*i]).numpy() #size of the weight tensor 
            
            shape_b = tf.shape(self.W[2*i+1]).numpy() # shape of the bias tensor
            size_b = tf.size(self.W[2*i+1]).numpy() #size of the bias tensor 
                        
            pick_w = parameters[0:size_w] #pick the weights 
            self.W[2*i].assign(tf.reshape(pick_w,shape_w)) # assign  
            parameters = np.delete(parameters,np.arange(size_w),0) #delete 
            
            pick_b = parameters[0:size_b] #pick the biases 
            self.W[2*i+1].assign(tf.reshape(pick_b,shape_b)) # assign 
            parameters = np.delete(parameters,np.arange(size_b),0) #delete 

            
    def loss_BC(self,x,y):

        loss_u = tf.reduce_mean(tf.square(y-self.evaluate(x)))
        return loss_u

    def loss_PDE(self, x_to_train_f):
    
        g = tf.Variable(x_to_train_f, dtype = 'float64', trainable = False)
    
        cost=10 
        sigma2=0.25

        x_f = g[:,0:1]
        y_f = g[:,1:2]
        t_f = g[:,2:3]

        with tf.GradientTape(persistent=True) as tape:

            tape.watch(x_f)
            tape.watch(y_f)
            tape.watch(t_f)

            g = tf.stack([x_f[:,0], y_f[:,0], t_f[:,0]], axis=1)   

            z = self.evaluate(g)
            p_x = tape.gradient(z,x_f)
            p_y = tape.gradient(z,y_f)

        p_t = tape.gradient(z,t_f)    
        p_xx = tape.gradient(p_x, x_f)
        p_yy = tape.gradient(p_y, y_f)
        p_xy = tape.gradient(p_x, y_f)
        p_yx = tape.gradient(p_y, x_f)

        del tape

        p=self.evaluate(g)

        f = p_t - sigma2/2*p_xx - sigma2/2*p_yy

        loss_f = tf.reduce_mean(tf.square(f))

        return loss_f
    
    def loss(self,x,y,g):

        loss_u = self.loss_BC(x,y)
        loss_f = self.loss_PDE(g)

        loss = loss_u + loss_f

        return loss, loss_u, loss_f
    
    def optimizerfunc(self,parameters):
        
        self.set_weights(parameters)
       
        with tf.GradientTape() as tape:
            tape.watch(self.trainable_variables)
            
            loss_val, loss_u, loss_f = self.loss(X_u_train, u_train, X_f_train)
            
        grads = tape.gradient(loss_val,self.trainable_variables)
                
        del tape
        
        grads_1d = [ ] #flatten grads 
        
        for i in range (len(layers)-1):

            grads_w_1d = tf.reshape(grads[2*i],[-1]) #flatten weights 
            grads_b_1d = tf.reshape(grads[2*i+1],[-1]) #flatten biases

            grads_1d = tf.concat([grads_1d, grads_w_1d], 0) #concat grad_weights 
            grads_1d = tf.concat([grads_1d, grads_b_1d], 0) #concat grad_biases

        return loss_val.numpy(), grads_1d.numpy()
    
    def optimizer_callback(self,parameters):
               
        loss_value, loss_u, loss_f = self.loss(X_u_train, u_train, X_f_train)
        
        u_pred = self.evaluate(X_u_test)
        error_vec = np.linalg.norm((u-u_pred),2)/np.linalg.norm(u,2)
        
        tf.print(self.itera, loss_value, loss_u, loss_f, error_vec)
        self.itera +=1

# *Solution Plot*

In [202]:
def solutionplot(u_pred,X_u_train,u_train):
    
    fig, ax = plt.subplots()
    ax.axis('off')

    gs0 = gridspec.GridSpec(2, 3)
    gs0.update(top=1, bottom=0, left=0.1, right=2, wspace=0.3, hspace =0.4)
    ax = plt.subplot(gs0[0, :])

    h = ax.imshow(u_pred, interpolation='nearest', cmap='rainbow', 
                extent=[T.min(), T.max(), X.min(), X.max()], 
                origin='lower', aspect='auto')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(h, cax=cax)
    
    ax.plot(X_u_train[:,1], X_u_train[:,0], 'kx', label = 'Data (%d points)' % (u_train.shape[0]), markersize = 4, clip_on = False)

    line = np.linspace(x.min(), x.max(), 2)[:,None]
    #ax.plot(t[25]*np.ones((2,1)), line, 'w-', linewidth = 1)
    #ax.plot(t[50]*np.ones((2,1)), line, 'w-', linewidth = 1)
    #ax.plot(t[75]*np.ones((2,1)), line, 'w-', linewidth = 1)    

    ax.set_xlabel('$t$')
    ax.set_ylabel('$x$')
    ax.legend(frameon=False, loc = 'best')
    ax.set_title('$u(x,t)$', fontsize = 10)
    
    ''' 
    Slices of the solution at points t = 0.25, t = 0.50 and t = 0.75
    '''
    
    ####### Row 1: u(t,x) slices ##################
    #gs1 = gridspec.GridSpec(1, 3)
    #gs1.update(top=0.3, bottom=-0.1, left=0.1, right=2, wspace=0.5)

    ax = plt.subplot(gs0[1, 0])
    #ax.plot(x,usol.T[0,:], 'b-', linewidth = 2, label = 'Exact')       
    ax.plot(x,u_pred.T[0,:], 'r', linewidth = 2, label = 'Prediction')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$u(x,t)$')    
    ax.set_title('$t = 0.s$', fontsize = 10)
    #ax.axis('square')
    ax.set_xlim([-1.1,1.1])
    ax.set_ylim([-0.1,9])

    ax = plt.subplot(gs0[1, 1])
    #ax.plot(x,usol.T[50,:], 'b-', linewidth = 2, label = 'Exact')       
    ax.plot(x,u_pred.T[500,:], 'r', linewidth = 2, label = 'Prediction')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$u(x,t)$')
    #ax.axis('square')
    ax.set_xlim([-1.1,1.1])
    ax.set_ylim([-0.1,9])
    ax.set_title('$t = 0.1s$', fontsize = 10)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.35), ncol=5, frameon=False)

    ax = plt.subplot(gs0[1, 2])
    #ax.plot(x,usol.T[75,:], 'b-', linewidth = 2, label = 'Exact')       
    ax.plot(x,u_pred.T[750,:], 'r', linewidth = 2, label = 'Prediction')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$u(x,t)$')
    #ax.axis('square')
    ax.set_xlim([-1.1,1.1])
    ax.set_ylim([-0.1,9])    
    ax.set_title('$t = 0.15s$', fontsize = 10)
    
    #plt.tight_layout()
    plt.savefig('Ornstein-Uhlenbeck.png',dpi = 500)   

# *Model Training and Testing*

A function '**model**' is defined to generate a NN as per the input set of hyperparameters, which is then trained and tested. The L2 Norm of the solution error is returned as a comparison metric

In [203]:
N_u = 20000 #Total number of data points for 'u'
N_f = 10000 #Total number of collocation points 

# Training data
X_f_train, X_u_train, u_train = trainingdata(N_u, N_f)

layers = np.array([3,20,20,20,20,20,20,20,20,1]) #8 hidden layers

PINN = Sequentialmodel(layers)

init_params = PINN.get_weights().numpy()

start_time = time.time() 

# train the model with Scipy L-BFGS optimizer
results = scipy.optimize.minimize(fun = PINN.optimizerfunc, 
                                  x0 = init_params, 
                                  args=(), 
                                  method='L-BFGS-B', 
                                  jac= True,        # If jac is True, fun is assumed to return the gradient along with the objective function
                                  callback = PINN.optimizer_callback, 
                                  options = {'disp': None,
                                            'maxcor': 200, 
                                            'ftol': 1 * np.finfo(float).eps,  #The iteration stops when (f^k - f^{k+1})/max{|f^k|,|f^{k+1}|,1} <= ftol
                                            'gtol': 5e-8, 
                                            'maxfun':  50000, 
                                            'maxiter': 500,
                                            'iprint': -1,   #print update every 50 iterations
                                            'maxls': 50})

elapsed = time.time() - start_time                
print('Training time: %.2f' % (elapsed))

print(results)

PINN.set_weights(results.x)

''' Model Accuracy ''' 
u_pred = PINN.evaluate(X_u_test)

error_vec = np.linalg.norm((u-u_pred),2)/np.linalg.norm(u,2)        # Relative L2 Norm of the error (Vector)
print('Test Error: %.5f'  % (error_vec))

#u_pred = np.reshape(u_pred,(256,1000),order='F')                        # Fortran Style ,stacked column wise!

''' Solution Plot '''
#solutionplot(u_pred,X_u_train,u_train)

0 0.16543889208460588 0.14839009849999288 0.017048793584612996 3.146861144478743
1 0.15549987582466371 0.14186042166451607 0.013639454160147647 2.9053260173430426
2 0.13560205116373353 0.11946805716288422 0.016133994000849295 2.1058710551622
3 0.12445719170923168 0.10546837475069824 0.018988816958533435 1.6849843031734364
4 0.11218970177117685 0.097935124939692475 0.014254576831484367 1.5487643080944204
5 0.0997397760308839 0.0923889200198615 0.007350856011022389 1.5550804232090358
6 0.094728275290200462 0.091136829472666339 0.0035914458175341168 1.5037651748457672
7 0.092575545297205158 0.090621603423832964 0.001953941873372193 1.3895493050120886
8 0.091916709766173721 0.090433857094243286 0.0014828526719304349 1.352706680486614
9 0.091166419679658772 0.090215591862166392 0.00095082781749238239 1.3611849164382583
10 0.090449790902860122 0.089644342766648941 0.00080544813621118052 1.3180796682372802
11 0.090248224581982034 0.0896071498847097 0.0006410746972723302 1.3242108034480118
12 

97 0.076737721263952743 0.076138915128862783 0.00059880613508996061 2.530120074662394
98 0.076719840655358168 0.076094575780009513 0.00062526487534864919 2.509187281958041
99 0.076703972426968178 0.076101603760955164 0.00060236866601301713 2.5043782788875646
100 0.076676093422583011 0.076096778475549631 0.00057931494703338025 2.4910161459925666
101 0.076625384737220886 0.076060486103668487 0.00056489863355240237 2.4899156078375424
102 0.076578562052030369 0.076056670806608037 0.00052189124542233189 2.4647465389779004
103 0.076543375515357281 0.076047996048053637 0.00049537946730365129 2.4686218465598
104 0.076516653548566849 0.0760396747633187 0.00047697878524814583 2.489244121754386
105 0.0764914736507909 0.07602170192359943 0.00046977172719146548 2.5119116159948778
106 0.076449176596450327 0.075988740722062839 0.00046043587438748216 2.5618968605038916
107 0.076416776432968792 0.075894722218918811 0.00052205421404997944 2.583599459144498
108 0.0763970437580607 0.075910712153239232 0.0

192 0.075532523023916345 0.075375825836225291 0.00015669718769105765 2.6005225273740384
193 0.075526009339091787 0.075386118149855685 0.00013989118923610471 2.5996320003195663
194 0.075521687154718561 0.075367880299086351 0.00015380685563221329 2.59278727797711
195 0.0755172365116866 0.075370321016074132 0.00014691549561245847 2.5964783404875207
196 0.07551371095350945 0.07534175254330347 0.00017195841020598357 2.5931325424470453
197 0.075510181296752008 0.075338969544257589 0.0001712117524944249 2.5989636276945576
198 0.075508831550405733 0.075324963775199816 0.00018386777520591174 2.593637285413928
199 0.0755035658690728 0.075328481577521339 0.00017508429155145238 2.589628560315368
200 0.075500405683393543 0.07530899668724865 0.00019140899614489024 2.5940673425400664
201 0.07549581234771531 0.0753116243017601 0.00018418804595520802 2.591006967586078
202 0.075489280702776537 0.075312670810692026 0.00017660989208451646 2.587708686470651
203 0.075481784467549079 0.075302642659615338 0.0

288 0.033224054862811825 0.030088575021410588 0.00313547984140124 6.856457140082586
289 0.033009506746635081 0.029850008576128847 0.0031594981705062334 6.739316540333424
290 0.032800860273138159 0.029737156574578025 0.0030637036985601347 6.789402346584521
291 0.032678416426103217 0.029669975133249319 0.0030084412928538949 6.801336042070293
292 0.032296265212995875 0.029491687288152945 0.0028045779248429335 6.803784696198196
293 0.031676575867618018 0.02915619966329229 0.0025203762043257256 6.774252245862126
294 0.031120431508039222 0.028548532286407465 0.0025718992216317562 6.817172683571155
295 0.030798098143916808 0.028027282729574426 0.002770815414342384 6.870305182112495
296 0.030609584373411231 0.027802034780596546 0.0028075495928146844 6.979094659714217
297 0.030488310857617121 0.027745523850818393 0.0027427870067987265 6.99738672226791
298 0.030316156637841714 0.027573610702156304 0.002742545935685411 7.09435899836124
299 0.030123606309569829 0.027343014500771107 0.0027805918087

384 0.0077241106074728075 0.0045589998270431763 0.0031651107804296307 7.865904265119134
385 0.0075735129788262259 0.0045272691696523031 0.0030462438091739224 7.828376351470144
386 0.0074562515588004604 0.00432195687477691 0.0031342946840235506 7.85870990156909
387 0.0073592500696384212 0.004111095932717938 0.0032481541369204837 7.730060971567121
388 0.0072566296097395146 0.0039441713433985273 0.0033124582663409878 7.755703764580456
389 0.0071742561725913557 0.0040802052803326216 0.003094050892258734 7.781463792416886
390 0.0071120323629946585 0.0039619108919613615 0.0031501214710332971 7.788854616176025
391 0.0070232394754631746 0.0037727651565619262 0.0032504743189012489 7.785581483370472
392 0.0067331568116643752 0.0035148359340493249 0.00321832087761505 7.836752117968198
393 0.0064741449538600962 0.0032211107572552508 0.003253034196604845 7.844785908218829
394 0.0063034595171252272 0.0032726406585786151 0.0030308188585466116 7.9028373136740555
395 0.0061557237091365916 0.00337587277

477 0.00089358269402238879 0.00020209417147245564 0.0006914885225499331 7.830172948922719
478 0.00087235136622642966 0.00019364253035537304 0.0006787088358710566 7.834093566312121
479 0.0008398642288462699 0.00018112580653682246 0.00065873842230944747 7.823868423916274
480 0.000818574149398006 0.000172214329515481 0.000646359819882525 7.816517659271849
481 0.00080405458642240956 0.00016864376278502452 0.00063541082363738506 7.82435711467675
482 0.00079377004836762291 0.00016856860990284027 0.00062520143846478264 7.815316789026961
483 0.00078528154045220966 0.00016996284489124904 0.00061531869556096061 7.821346006475029
484 0.00077734009769265112 0.00016405062030196615 0.000613289477390685 7.8308003528554
485 0.000769539126250746 0.00016396336084918323 0.00060557576540156278 7.828668387872923
486 0.000758200593686514 0.00016054492129981507 0.00059765567238669894 7.837127571651328
487 0.00074592169888574244 0.00015851843680762855 0.00058740326207811389 7.838764252212222
488 0.00073161042

' Solution Plot '

In [204]:
u_pred_2 = np.reshape(u_pred,(128, 128, len(t)), order='C')   
u_pred.shape

TensorShape([1638400, 1])

In [205]:
from matplotlib import cm

def plot_3d_ic(time):
    fig = plt.figure(figsize=(10, 10))
    ax = plt.axes(projection='3d')
    ax.plot_surface(X[:,:,0], Y[:,:,0], test[:,:,time],cmap=cm.inferno)
    ax.set_zlim(0, 2)

    

    
def plot_3d_pred(time):
    fig = plt.figure(figsize=(10, 10))
    ax = plt.axes(projection='3d')
    ax.plot_surface(X[:,:,0], Y[:,:,0], u_pred_2[:,:,time],cmap=cm.inferno)
    ax.set_zlim(0, 2)
    
    
    
def plot_2d(time):
    plt.plot(y, u_pred_2[:,64,time])
    plt.plot(y, u_pred_2[64,:,time])
    #plt.ylim(0, 2)
    #plt.xlim(-1, 1)

In [206]:
import ipywidgets

ipywidgets.interact(plot_3d_ic, time=(0, 99, 3))
ipywidgets.interact(plot_3d_pred, time=(0, 99, 3))

interactive(children=(IntSlider(value=48, description='time', max=99, step=3), Output()), _dom_classes=('widge…

interactive(children=(IntSlider(value=48, description='time', max=99, step=3), Output()), _dom_classes=('widge…

<function __main__.plot_3d_pred(time)>